In [ ]:
!cat container/Dockerfile

In [2]:
print("This is a test.")
print("this really is a test.")

This is a test.
this really is a test.


In [97]:
%%sh

# The name of our algorithm
algorithm_name=my-test-nn

cd container

chmod +x nn/train
chmod +x nn/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  37.89kB
Step 1/25 : FROM ubuntu:16.04
 ---> 2a697363a870
Step 2/25 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 28b035530484
Step 3/25 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 609870c850b5
Step 4/25 : RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py
 ---> Using cache
 ---> 69c32e7c3c31
Step 5/25 : RUN pip install numpy
 ---> Using cache
 ---> e11ac61f9717
Step 6/25 : RUN pip install scipy
 ---> Using cache
 ---> 58d65875a34d
Step 7/25 : RUN pip install scikit-learn
 ---> Using cache
 ---> cd4ee06bf927
Step 8/25 : RUN pip install pandas
 ---> Using cache
 ---> 62f3023e03b9
Step 9/25 : RUN pip install flask
 ---> Using cache
 ---> 25381709b7ac
Step 10/25 : RUN pip install gevent
 ---> Using cache
 ---> 130187e4c92e
Step 11/25 : RUN 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [11]:
#!aws s3 cp s3://f404466-emr-poc/data/tweets/tweets3.csv ./CNN/data/tweets2.csv
#!mkdir /tmp/CNN/train
#!aws s3 cp s3://f404466-emr-poc/data/tweets/train/tweets.csv /tmp/CNN/train/tweets.csv
#!aws s3 cp s3://f404466-emr-poc/data/tweets/train/tweets.csv ./data/train/tweets.csv
#!aws s3 cp s3://f404466-emr-poc/data/tweets/test/tweets.csv ./data/test/tweets.csv
#!mkdir /tmp/CNN/test
#!aws s3 cp s3://f404466-emr-poc/data/tweets/test/tweets.csv /tmp/CNN/test/tweets.csv
#!aws s3 cp s3://f404466-emr-poc/data/tweets/tweets3.csv ./CNN/data/tweets.csv

In [98]:
import pandas as pd

df = pd.read_csv("/tmp/NN/train/tweets.csv")
df = df.drop(['Unnamed: 0'], axis=1)

df=df[df['sentiment']!='MIXED']

train = df

df = pd.read_csv("/tmp/NN/test/tweets.csv")
df = df.drop(['Unnamed: 0'], axis=1)

df=df[df['sentiment']!='MIXED']

test = df

data = pd.concat([train, test])

data.to_csv('data/tweets.csv')

In [102]:
# S3 prefix
prefix = 'f404466'

# Define IAM role
import boto3
import re
import datetime
import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

role = get_execution_role()

WORK_DIRECTORY = 'data/train'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix="{}/{}".format(prefix, WORK_DIRECTORY))
#data_location = 'file:///tmp/CNN/'
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'227919005974.dkr.ecr.us-east-2.amazonaws.com/my-test-nn:latest'
#image = f'my-test-nn:latest'

#instance_type = 'local'
instance_type = 'ml.p3.2xlarge'
#instance_type = 'ml.m5.2xlarge'

classifier = sage.estimator.Estimator(
    image, 
    role = role, 
    train_instance_count = 1, 
    train_instance_type = instance_type, 
    #image_name = image)
    output_path="s3://f404466-emr-poc/output",
    sagemaker_session=sess)

classifier.fit(data_location)

2019-05-29 19:27:51 Starting - Starting the training job...
2019-05-29 19:27:53 Starting - Launching requested ML instances......
2019-05-29 19:29:00 Starting - Preparing the instances for training...
2019-05-29 19:29:46 Downloading - Downloading input data...
2019-05-29 19:29:54 Training - Downloading the training image..
Using TensorFlow backend.
Starting the training.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/25
 - 6s - loss: 0.4743 - acc: 0.7829
Epoch 2/25

2019-05-29 19:30:26 Training - Training image download completed. Training in progress. - 6s - loss: 0.3842 - acc: 0.8574
Epoch 3/25
 - 6s - loss: 0.3837 - acc: 0.8574
Epoch 4/25
 - 6s - loss: 0.3830 - acc: 0.8574
Epoch 5/25
 - 5s - loss: 0.3839 - acc: 0.8574
Epoch 6/25
 - 5s - loss: 0.3836 - acc: 0.8574
Epoch 7/25
 - 6s - loss: 0.

In [103]:
# deploy it to an endpoint
predictor = classifier.deploy(1, 'ml.m5.2xlarge', endpoint_name ='demo-test-endpoint')

# connect to the endpoint
predictor = sage.predictor.RealTimePredictor(
    'demo-test-endpoint',
    sagemaker_session=sess,
    content_type="text/csv"
)       

--------------------------------------------------------------------------------------!

In [105]:
test1 = ["This is a great movie! This is a great movie! This is a great movie!","POSITIVE"]
test2 = ["This is a terrible movie!","NEGATIVE"]
test3 = ["This is a movie!","NEUTRAL"]

inputs = [test1,test2,test3]

input_data = "\n".join(
    [",".join(l) for l in inputs]
)
input_data
#print(input_data)
#predictor.predict(input_data)
predictor.delete_endpoint()

In [80]:

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def transform_data(dataset):
    
    print(dataset)
    
    tokenizer_obj = Tokenizer()
    
    tokenizer_obj.fit_on_texts(dataset['tweet'])
    
    print("{} Records in file...".format(len(dataset)))
    
    max_length = max([len(s.split()) for s in dataset['tweet']])
    
    print("Max Length {}".format(max_length))
    
    vocab_size = len(tokenizer_obj.word_index)+1
    
    print("Vocab Size {}".format(vocab_size))
    
    x_train_tokens = tokenizer_obj.texts_to_sequences(dataset['tweet'])
    
    x_train_pad = pad_sequences(x_train_tokens, maxlen=max_length, padding='post')

    #y_train = np.array(dataset.sentiment)
    
    return x_train_pad #, y_train

#import StringIO
from io import StringIO
s = StringIO(input_data)
data = pd.read_csv(s, header=None)
data.columns = ['tweet','sentiment']
X = transform_data(data)
X

                                               tweet sentiment
0  This is a great movie! This is a great movie! ...  POSITIVE
1                          This is a terrible movie!  NEGATIVE
2                                   This is a movie!   NEUTRAL
3 Records in file...
Max Length 15
Vocab Size 7


array([[1, 2, 3, 5, 4, 1, 2, 3, 5, 4, 1, 2, 3, 5, 4],
       [1, 2, 3, 6, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [73]:
!pip install tensorflow

  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 92.5MB 584kB/s 
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 2.2MB 27.0MB/s 
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/d1/4f/950dfae467b384fc96bc6469de25d832534f6b4441033c39f914efd13418/astor-0.8.0-py2.py3-none-any.whl
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/da/3f/9b0355080b81b15ba6a9ffcf1f5ea39e307a2778b2f2dc8694724e8abd5b/absl-py-0.7.1.tar.gz
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/8a/48/a76be51647d0eb9f10e2a4511bf3ffb8cc1e6b14e9e4fab46173aa79f981/termcolor-1.1.0.tar.gz
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/4e/35/11749bf99b2d4e3cceb4d55ca22590b0d7c2c62b9de38ac4a4a7f4687421/gas

In [69]:
df = pd.read_csv("./data/test/tweets.csv")
df = df.drop(['Unnamed: 0'], axis=1)

X_test = (
  df.head().values
)
input_data = "\n".join(
    [",".join(l) for l in X_test.astype('str').tolist()]
)

#input_data = '0,create new memories in this beautiful home,POSITIVE'
predictor.predict(input_data)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/demo-test-endpoint in account 227919005974 for more information.

In [137]:
from __future__ import print_function
import sys
print(sys.version)
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


df = pd.read_csv("./data/train/tweets.csv")
df = df.drop(['Unnamed: 0'], axis=1)

X_test = (
  df.values
)
input_data = "\n".join(
    [",".join(l) for l in X_test.astype('str').tolist()]
)

s = StringIO(input_data)
dataset = pd.read_csv(s, header=None)

dataset.columns = ['tweet','sentiment']
tokenizer_obj = Tokenizer()   

tokenizer_obj.fit_on_texts(dataset['tweet'])    
max_length = max([len(s.split()) for s in dataset['tweet']])    
vocab_size = len(tokenizer_obj.word_index)+1    
x_train_tokens = tokenizer_obj.texts_to_sequences(dataset['tweet'])
x_train_pad = pad_sequences(x_train_tokens, maxlen=69)

model = load_model('/tmp/tmpxdqiu9as/model/ann-churn.h5')
predictions = model.predict(x_train_pad)

3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]


In [111]:


encoder = LabelEncoder()
encoder.fit(dataset['sentiment'])
encoded_Y = encoder.transform(dataset['sentiment'])
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

print(dummy_y)

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [28]:
sess.delete_endpoint(predictor.endpoint)

In [86]:
!find /tmp/* -name "*churn.h5"

/tmp/tmp2q__gol6/model/ann-churn.h5
/tmp/tmp36qpak9f/ann-churn.h5
/tmp/tmp4bcw3gu3/ann-churn.h5
/tmp/tmp70zyg0a0/ann-churn.h5
/tmp/tmp78nuu9mt/ann-churn.h5
/tmp/tmp85d6f_4y/model/ann-churn.h5
/tmp/tmpgkadbvzb/model/ann-churn.h5
/tmp/tmpri_loh0n/model/ann-churn.h5
/tmp/tmpusowdard/model/ann-churn.h5
/tmp/tmpv1mj8re3/model/ann-churn.h5
/tmp/tmpxdqiu9as/model/ann-churn.h5
/tmp/tmpytcqdab2/model/ann-churn.h5


In [166]:
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Activation
from keras.layers.embeddings import Embedding
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=69, init='normal', activation='relu'))
    model.add(Dense(4, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

df = pd.read_csv("./data/train/tweets.csv")
df = df.drop(['Unnamed: 0'], axis=1)

x_train = df['tweet']
y_train = df['sentiment']
y_train

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y

tokenizer_obj = Tokenizer()   

tokenizer_obj.fit_on_texts(x_train)    
max_length = max([len(s.split()) for s in x_train])    
vocab_size = len(tokenizer_obj.word_index)+1    
x_train_tokens = tokenizer_obj.texts_to_sequences(x_train)
x_train_pad = pad_sequences(x_train_tokens, maxlen=69, padding='post')

estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=200, batch_size=5, verbose=0)
#X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=seed)
print(x_train_pad)
print(dummy_y)
estimator.fit(x_train_pad, dummy_y)

df = pd.read_csv("./data/test/tweets.csv")
df = df.drop(['Unnamed: 0'], axis=1)

x_test = df['tweet']
y_test = df['sentiment']

tokenizer_obj.fit_on_texts(x_test)    
max_length = max([len(s.split()) for s in x_test])    
vocab_size = len(tokenizer_obj.word_index)+1    
x_test_tokens = tokenizer_obj.texts_to_sequences(x_test)
x_test_pad = pad_sequences(x_test_tokens, maxlen=69, padding='post')

encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y

predicted = estimator.predict(x_test_pad)

[[427   1  43 ...   0   0   0]
 [  5  23 713 ...   0   0   0]
 [ 13  89  12 ...   0   0   0]
 ...
 [179  10 123 ...   0   0   0]
 [ 13 990 487 ...   0   0   0]
 [ 21  14  96 ...   0   0   0]]
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, input_dim=69, activation="relu", kernel_initializer="normal")`
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="sigmoid", kernel_initializer="normal")`


In [168]:
print(encoder.inverse_transform(predicted))

['NEUTRAL' 'NEGATIVE' 'NEUTRAL' ... 'NEUTRAL' 'NEUTRAL' 'NEUTRAL']


In [179]:
from sklearn.cross_validation import train_test_split

raw_data = pd.read_csv("./data/train/tweets.csv")

X = raw_data['tweet']

x_train = raw_data['tweet']
y_train = raw_data['sentiment']
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y

X_train, X_test, Y_train, Y_test = train_test_split(x_train, dummy_y, test_size=0.33, random_state=seed)


ModuleNotFoundError: No module named 'sklearn.cross_validation'